# Anomaly calculations

In [ ]:
# Imports
from earthkit import transforms as ekt
from earthkit import data as ekd
from earthkit.data.testing import earthkit_remote_test_data_file
ekd.settings.set("cache-policy", "user")

## Load some test data

In this example we will use hourly ERA5 2m temperature data on a 0.5x0.5 spatial grid for the year 2015 as
our physical data; and we will use the NUTS geometries which are stored in a geojson file.

All `earthkit-transforms` methods can be called with `earthkit-data` objects (Readers and Wrappers) or with a pre-loaded `xarray`. To reduce the number of conversions in the example, we will convert to xarray in the first cell and use that data object for all subsequent steps.

In [ ]:
# Get some demonstration ERA5 data, this could be any url or path to an ERA5 grib or netCDF file.
remote_era5_file = earthkit_remote_test_data_file("era5_temperature_france_2015_2016_2017_3deg.grib")
era5_data = ekd.from_source("url", remote_era5_file)

# convert to xarray to save repeated conversion in further steps
era5_xr = era5_data.to_xarray(time_dim_mode="valid_time")
era5_xr

## Calculate the daily climatology of the ERA5 data

In [ ]:
climatology_daily_mean = ekt.climatology.daily_mean(era5_xr)
climatology_daily_mean

## Calculate the anomaly and relative anomaly

In [ ]:
anomaly = ekt.climatology.anomaly(era5_xr, climatology_daily_mean)
anomaly

In [ ]:
relative_anomaly = ekt.climatology.relative_anomaly(
    era5_xr, climatology_daily_mean
)
relative_anomaly

## Plot the output for a random location

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime

start, end = datetime(2015,1,1), datetime(2015,3,31)

isel_kwargs = {"latitude":2, "longitude":4}
sel_kwargs = {"valid_time": slice(start, end)}

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,5))

for data in [anomaly, relative_anomaly]:
    var_name = list(data.data_vars.keys())[0]
    p_data = data[var_name].isel(**isel_kwargs).sel(**sel_kwargs)
    p_data.plot(ax=ax, label=var_name)

ax.set_xlim(start, end)
ax.set_ylabel("2m temperature anomaly [K] and relative anomaly [%]")
ax.hlines(0, xmin=start, xmax=end, color="black", linestyle="--")
ax.legend()

